# DataFrames with data from JSON files

In [1]:
import h5py
import ujson
import gzip
import ast
from pandas import DataFrame
from toolz import dissoc
from toolz import dissoc, partition_all
from castra import Castra
import time
import datetime
import dask.dataframe as dd
import dask.bag as db
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
from pandas import HDFStore

In [31]:
path_to_file = "C:\\dtu\\ctbd\\amazon_dataset\\data.hdf5"
path_to = "C:\\dtu\\ctbd\\amazon_dataset\\"
f = 'Office_Products_5.json'
reviews_columns = ['asin', 'reviewerID', 'overall','summary','reviewText','reviewTime','unixReviewTime']
metadata_columns = ['asin','title','price','imUrl','related','also_bought','also_viewed','bought_together','salesRank','brand','categories']
str_dt = h5py.special_dtype(vlen=unicode)
#dtypes = np.dtype([('asin',str_dt),('reviewerID',str_dt),('reviewerName',str_dt),('overall',float),('summary',str_dt),
 #                  ('reviewText',str_dt),('reviewTime',str_dt),('unixReviewTime',int)])
chunksize = 5000
current_index = 0

#Convert a line of JSON into a cleaned up dict.
def to_json(line):
    return dissoc(ujson.loads(line.encode('utf8')),'reviewerName')

#Convert a not proper line of JSON (due to single quotes) into a cleaned up dict.
def fix_json(line):
    return ast.literal_eval(line)

#Convert a list of JSON strings into a DataFrame
def to_df(batch,filename):
    if filename == 'metadata':
        blobs = map(fix_json,batch)
        df = DataFrame.from_records(blobs, columns=metadata_columns)
    else:
        blobs = map(to_json, batch)
        df = DataFrame.from_records(blobs, columns=reviews_columns)
    return df

def create_new(path_to_file,df, current_index):
    types = df.apply(lambda x: pd.lib.infer_dtype(x.values))
    for col in types[types=='unicode'].index:
        df[col] = df[col].astype(str)
    print df.apply(lambda x: pd.lib.infer_dtype(x.values))
    df.to_hdf(path_to_file,'/amazon',format='table',data_columns=True,complevel=9, complib='bzip2',min_itemsize={'summary':150,'reviewText':35000})
    current_index+=chunksize
    return 'blabla', current_index

def extend_hdf5(path_to_file,df, current_index, chunksize):
    types = df.apply(lambda x: pd.lib.infer_dtype(x.values))
    for col in types[types=='unicode'].index:
        df[col] = df[col].astype(str)
    df.to_hdf(path_to_file,'/amazon',format='table',data_columns=True,append=True)
    current_index+=chunksize
    return current_index

def create_hdf5(fullpath,chunksize,current_index):
    filename = fullpath.split('/')[-1].split('.')[0]
    with open(fullpath,'r') as f:
        batches = partition_all(chunksize, f)
        store = None
        for batch in batches:
            df = to_df(batch,filename)
            if store==None:
                store, current_index = create_new(path_to_file,df,current_index)
            else:
                current_index = extend_hdf5(path_to_file,df, current_index, chunksize)

In [32]:
create_hdf5(path_to+f,chunksize,current_index)

#Check status
with h5py.File(path_to_file, "r+") as hf:
    data = hf.get('amazon/table')
    print data.shape
    print data[:10]

asin                string
reviewerID          string
overall           floating
summary             string
reviewText          string
reviewTime          string
unixReviewTime     integer
dtype: object
(53258,)
[ (0L, 'B00000JBLH', 'A32T2H8150OJLU', 5.0, 'A solid performer, and long time friend', "I bought my first HP12C in about 1984 or so, and it served me faithfully until 2002 when I lost it while travelling.  I searched for another one to replace it, but found one difficult to come by in my area.  So, I decided to buy up and purchased an HP 49G.  What a mistake!  I know that many people view the HP 49G (now 49G+) as the flagship of the HP line, but for me that was a disaster.The 49G may be powerful, but using it is extremely counterintuitive...and the manual was sketchy at best.  The 12C, on the other hand, does what I need in a way that makes good sense to me.If you are looking for a solid, reliable calculator, the HP12C may be for you.  It's programmable.  It does basic statisti

In [33]:
df = dd.read_hdf(path_to_file,'/amazon')
result = df.where(df['overall']<5).groupby('asin').mean().nlargest(10,columns='overall').compute()
result.reset_index()[['asin','overall']]

,asin,overall
0,B00002NDS2,4.0
1,B00004Z4A8,4.0
2,B00004Z5T0,4.0
3,B00004TS2L,4.0
4,B00004SUJT,4.0
5,B00005T407,4.0
6,B00004Z63V,4.0
7,B00004Z6NA,4.0
8,B00000JBLU,4.0
9,B00005QJED,4.0
